In [ ]:
#https://huggingface.co/docs/transformers/model_doc/bert

In [ ]:
!pip install evaluate
!pip install seqeval
!pip install datasets

1. что такое логи и логиты?  
2. как понять, какие аргументы мне нужны?
3. до какого качества надо добить бэйзлайн?
4.
5. что то типа теория

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

from datasets import load_dataset
from collections import Counter
import evaluate

from transformers import BertConfig, BertModel, BertTokenizerFast, BertForSequenceClassification, TrainingArguments, Trainer, get_linear_schedule_with_warmup, AdamW, get_cosine_schedule_with_warmup

In [ ]:
ds = load_dataset("ai-forever/kinopoisk-sentiment-classification")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('google-bert/bert-base-uncased')

In [ ]:
def slice_token(index, sentences, labels, tokenizer, max_length):
    start, stop, step = index.indices(len(sentences))
    result = []
    for i in range(start, stop, step):
        encoding = tokenizer(
                sentences[i],
                padding='max_length',
                truncation = True,
                max_length = max_length,
                return_tensors = 'pt'
            )
        encoding['labels'] = [labels[i]]
        result.append({key : value[0] for key, value in encoding.items()})

    return result

In [ ]:
class NERDataset(Dataset):
    def __init__(self, sentences, labels, tokenizer, max_length):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):

        if isinstance(idx, slice):
            # Обработка среза
            return slice_token(idx, self.sentences, self.labels, self.tokenizer, self.max_length)
        elif isinstance(idx, int):
            tokens = self.sentences[idx]
            tag = self.labels[idx]
            print(tokens, tag)
            # токенизируем
            encoding = self.tokenizer(
                tokens,
                padding='max_length',
                truncation = True,
                max_length = self.max_length,
                return_tensors = 'pt'
            )
            encoding['labels'] = [tag]
            return {key : value[0] for key, value in encoding.items()}

In [ ]:
dataset_train = NERDataset(ds['train']['text'], ds['train']['label'], tokenizer, 512)
dataset_test = NERDataset(ds['test']['text'], ds['test']['label'], tokenizer, 512)
dataset_val = NERDataset(ds['validation']['text'], ds['validation']['label'], tokenizer, 512)

In [ ]:
little_train_loader = DataLoader(dataset_train[0:5250], batch_size=32)
little_val_loader = DataLoader(dataset_val[0:750], batch_size = 32)

In [ ]:
test_loader = DataLoader(dataset_test, batch_size=32, pin_memory=True)
train_loader = DataLoader(dataset_train, batch_size=32)
val_loader = DataLoader(dataset_val, batch_size = 32)

In [ ]:
# Initializing a model (with random weights) from the google-bert/bert-base-uncased style configuration
model = BertForSequenceClassification.from_pretrained('google-bert/bert-base-uncased', num_labels = len(set(ds['train']['label'])))
#num_labels = len(set(ds['train']['label']))

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), correct_bias=False) # а здесь можно не фильтровать?
total_steps = len(dataset_train) * 5 * 32

scheduler = get_cosine_schedule_with_warmup(
  optimizer,
  num_warmup_steps=total_steps*0.05,
  num_training_steps=total_steps
)

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    eval_strategy="epoch",    # Evaluate after each epoch
    learning_rate=1.8562022777790423e-05,             # Learning rate
    per_device_train_batch_size=32, # Batch size for training
    per_device_eval_batch_size=32,  # Batch size for evaluation
    num_train_epochs=5,             # Number of epochs
    weight_decay=0.001928128030703447,              # Strength of weight decay
    logging_dir="./logs",           # Directory for storing logs
    logging_steps=10,               # что это такое?
    save_strategy="epoch",          # Save model after each epoch
    load_best_model_at_end=True,    # Load the best model after training
    metric_for_best_model="f1", # Use F1 score to choose the best model
    seed=224
)

In [ ]:
def compute_metrics(p):

    predictions, labels = p
    print(predictions, labels)
    # логиты в индексы
    predictions = predictions.argmax(axis=-1)

    # пихнем в метрику и получим результат
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='macro')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
def hp_space_fn(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 1e-3,log=True),
        "weight_decay" : trial.suggest_float("weight_decay", 1e-5, 0.1, log=True) # логарифмический масштаб для lr
        }

def model_init():
    return BertForSequenceClassification.from_pretrained('google-bert/bert-base-uncased', num_labels = len(set(ds['train']['label'])))

In [ ]:
trainer = Trainer(
    model=model,
    #model_init = model_init,
    args=training_args,
    train_dataset=little_train_loader.dataset,  # Training dataset
    eval_dataset=little_val_loader.dataset,   # Evaluation dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler)
    )


<ipython-input-18-be7e97f11a5a>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.9 MB/s eta 0:00:00


In [ ]:
best_run = trainer.hyperparameter_search(
    hp_space=hp_space_fn,
    n_trials=5, # количество запусков с разными гиперпараметрами
    direction="maximize", # максимизируем f1
    backend="optuna" # backend для поиска по сетке
)

[I 2025-02-24 18:42:00,255] A new study created in memory with name: no-name-ec992fbe-b9d6-4e5d-a0ab-6375c9506657
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lizaolva123 (lizaolva123-rggu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


...

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.124100,1.097858,0.351515,0.351515,0.351515,0.351515
2,1.120200,1.099805,0.351515,0.351515,0.351515,0.351515
3,1.090500,1.099279,0.329293,0.329293,0.329293,0.329293


[I 2025-02-24 19:54:46,688] Trial 3 finished with value: 1.3171717171717172 and parameters: {'learning_rate': 0.0007959039239082345, 'weight_decay': 0.0011870453400170528}. Best is trial 2 with value: 1.4141414141414141.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


eval/accuracy,██▁
eval/f1,██▁
eval/loss,▁█▆
eval/precision,██▁
eval/recall,██▁
eval/runtime,▄█▁
eval/samples_per_second,▅▁█
eval/steps_per_second,▅▁█
train/epoch,▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/grad_norm,▆▃▅▄▅▄█▃▂▃▃▅▄▂▃▃▄▁▃▃▂▂▃▂▄▃▄▁▄▄▃▃


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.105600,1.099564,0.329293,0.329293,0.329293,0.329293
2,1.099500,1.094747,0.375758,0.375758,0.375758,0.375758
3,1.089900,1.090811,0.373737,0.373737,0.373737,0.373737


[I 2025-02-24 20:13:32,224] Trial 4 finished with value: 1.494949494949495 and parameters: {'learning_rate': 1.8562022777790423e-05, 'weight_decay': 0.001928128030703447}. Best is trial 4 with value: 1.494949494949495.


In [ ]:
best_run.hyperparameters
#{'learning_rate': 1.8562022777790423e-05, 'weight_decay': 0.001928128030703447}

NameError: name 'best_run' is not defined

In [ ]:
# собственно обучение - автоматически делает логи
trainer.train()

# оценим модельку
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")

# Сохраним, что получилось
trainer.save_model("./ner_model")
#b5a31e3a762dc4fdbd905c7a205899ee8116917a

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lizaolva123 (lizaolva123-rggu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.098600,1.098207,0.341333,0.265523,0.231310,0.347706
2,1.105100,1.079131,0.428000,0.390884,0.444034,0.423574


[[ 0.00046749  0.21961598  0.12087952]
 [-0.03553335  0.192562    0.15544751]
 [-0.04315472  0.17604083  0.14022285]
 ...
 [-0.05274145  0.07253872  0.12016419]
 [-0.01645893  0.12051017  0.11495606]
 [-0.06673492  0.08820871  0.13595843]] [1 1 1 1 0 1 1 0 2 2 0 2 2 1 1 2 2 0 2 1 2 2 0 1 1 0 2 1 1 1 1 1 1 2 2 1 1
 1 0 0 0 2 0 0 1 0 2 0 0 0 0 1 0 1 2 0 0 1 2 2 0 1 0 2 2 2 2 1 0 2 2 1 0 2
 1 2 1 1 0 1 1 2 1 2 2 1 1 1 1 0 1 2 0 0 0 0 2 0 0 0 0 1 0 1 0 1 2 1 1 2 2
 2 2 1 0 2 0 0 2 1 0 2 0 2 2 0 2 1 1 0 0 1 0 1 2 0 2 2 1 0 1 0 1 0 2 2 2 2
 2 2 0 0 0 0 2 2 0 2 2 2 0 1 0 0 0 1 2 0 0 1 0 2 2 1 1 2 2 2 0 2 2 0 0 0 1
 1 1 1 2 1 1 1 0 1 2 0 0 1 2 2 2 0 1 2 0 0 2 0 1 0 2 1 0 0 1 1 1 1 1 0 1 0
 0 2 1 0 0 1 0 2 0 0 2 0 0 1 0 0 2 1 2 1 1 2 0 1 0 0 2 1 2 1 1 0 2 0 2 0 1
 0 2 0 0 2 2 2 0 1 0 2 2 0 0 1 0 1 1 1 1 1 2 0 2 1 0 2 2 1 2 2 0 0 0 1 2 2
 2 2 0 0 0 2 1 0 1 0 2 2 2 0 0 2 1 0 1 0 2 1 1 2 1 0 1 2 1 0 1 2 2 0 1 0 1
 2 1 0 2 0 1 2 2 2 0 1 2 1 0 1 0 2 1 1 0 0 1 1 2 2 0 2 1 0 2 2 1 1 1 0 0 0
 0 2 1 1 2

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[[ 0.37310582  0.18517362 -0.09300851]
 [ 0.2947414   0.15488403 -0.00180222]
 [ 0.1043917   0.08972649  0.1436004 ]
 ...
 [ 0.15666664  0.06193187  0.094532  ]
 [ 0.30395177  0.07567859 -0.00928094]
 [ 0.11123661  0.0594676   0.12483993]] [1 1 1 1 0 1 1 0 2 2 0 2 2 1 1 2 2 0 2 1 2 2 0 1 1 0 2 1 1 1 1 1 1 2 2 1 1
 1 0 0 0 2 0 0 1 0 2 0 0 0 0 1 0 1 2 0 0 1 2 2 0 1 0 2 2 2 2 1 0 2 2 1 0 2
 1 2 1 1 0 1 1 2 1 2 2 1 1 1 1 0 1 2 0 0 0 0 2 0 0 0 0 1 0 1 0 1 2 1 1 2 2
 2 2 1 0 2 0 0 2 1 0 2 0 2 2 0 2 1 1 0 0 1 0 1 2 0 2 2 1 0 1 0 1 0 2 2 2 2
 2 2 0 0 0 0 2 2 0 2 2 2 0 1 0 0 0 1 2 0 0 1 0 2 2 1 1 2 2 2 0 2 2 0 0 0 1
 1 1 1 2 1 1 1 0 1 2 0 0 1 2 2 2 0 1 2 0 0 2 0 1 0 2 1 0 0 1 1 1 1 1 0 1 0
 0 2 1 0 0 1 0 2 0 0 2 0 0 1 0 0 2 1 2 1 1 2 0 1 0 0 2 1 2 1 1 0 2 0 2 0 1
 0 2 0 0 2 2 2 0 1 0 2 2 0 0 1 0 1 1 1 1 1 2 0 2 1 0 2 2 1 2 2 0 0 0 1 2 2
 2 2 0 0 0 2 1 0 1 0 2 2 2 0 0 2 1 0 1 0 2 1 1 2 1 0 1 2 1 0 1 2 2 0 1 0 1
 2 1 0 2 0 1 2 2 2 0 1 2 1 0 1 0 2 1 1 0 0 1 1 2 2 0 2 1 0 2 2 1 1 1 0 0 0
 0 2 1 1 2

KeyboardInterrupt: 

In [ ]:
def eval_model(model, data_loader, device):
  model = model.eval()

  all_preds = torch.tensor([], device=device)
  all_trues = torch.tensor([], device=device)

  with torch.no_grad():
    for d in data_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["labels"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      #print(outputs)
      preds = torch.argmax(outputs['logits'], axis=-1)
      all_preds = torch.cat((all_preds, preds), -1)
      all_trues = torch.cat((all_trues, targets), -1)

  precision, recall, f1, _ = precision_recall_fscore_support(all_trues.cpu(), all_preds.cpu(), average='macro')
  acc = accuracy_score(all_trues.cpu(), all_preds.cpu())
  return {
      'accuracy': acc,
      'f1': f1,
      'precision': precision,
      'recall': recall
  }

In [ ]:
test = eval_model(model, test_loader, device)

In [ ]:
test

{'accuracy': 0.498,
 'f1': 0.49186082461483105,
 'precision': 0.4945688694129702,
 'recall': 0.498}

In [ ]:
precision, recall, f1, _ = precision_recall_fscore_support(torch.tensor(ds['test']['label']), torch.zeros(1500), average='macro')
acc = accuracy_score(torch.tensor(ds['test']['label']), torch.zeros(1500))
print({
      'accuracy': acc,
      'f1': f1,
      'precision': precision,
      'recall': recall})

{'accuracy': 0.3333333333333333, 'f1': 0.16666666666666666, 'precision': 0.1111111111111111, 'recall': 0.3333333333333333}


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True) # Подключите Google Drive
!cp -r /content/results /content/drive/MyDrive/чекпоинты # Замените на ваши пути

Mounted at /content/drive
